In [1]:
# Q1b
def expected_winnings(N, M):
    if M <= 0:
        return 0
    
    if N <= 1:
        return N
    
    cur_return = expected_return(1, N)
    M -= 1
    while M > 0:
        cur_return = cur_return * int(cur_return) / float(N) + (1 - int(cur_return) / float(N)) * expected_return(int(cur_return) + 1, N)
        M -= 1
        
    return cur_return
    
def expected_return(begin, end):
    return sum(range(begin, end + 1)) / float(end - begin + 1)

In [2]:
# Test 1
win = expected_winnings(6, 3)
win

4.666666666666667

In [3]:
# Test 2
win = expected_winnings(1, 3)
win

1

In [4]:
# Test 3
win = expected_winnings(10, 1)
win

5.5

In [5]:
# Test 4
win = expected_winnings(10, 4)
win

7.915

In [6]:
# Q2
# Step 1. Check the data 
import pandas as pd
data = pd.read_csv('../Data/data_Problem2 (7).csv')
data.head()

,FROM_NODE,TO_NODE,VALUE
0,3.0,76,271791.828330
1,76.0,88,1458.625174
2,76.0,96,86848.361600
3,2.0,76,406695.000000
4,76.0,98,3227.734868


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 3 columns):
FROM_NODE    166 non-null float64
TO_NODE      167 non-null int64
VALUE        167 non-null float64
dtypes: float64(2), int64(1)
memory usage: 4.0 KB


In [8]:
data.describe()

,FROM_NODE,TO_NODE,VALUE
count,166.000000,167.000000,1.670000e+02
mean,82.012048,125.497006,1.419764e+07
std,51.458249,35.832756,5.810345e+07
min,1.000000,10.000000,2.158195e+02
25%,45.250000,94.500000,2.098236e+04
50%,76.000000,133.000000,9.058779e+04
75%,123.250000,157.000000,2.846257e+05
max,178.000000,179.000000,5.000000e+08


In [21]:
# Step 2. Build the algorithm
import csv
from collections import defaultdict
graph = defaultdict(list)
val_graph = defaultdict(list)

def build_graph():
    with open('../Data/data.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        header = next(csv_reader)
        for row in csv_reader:
            from_node, to_node, value = row

            try:
                from_node = int(float(from_node))
                to_node = int(float(to_node))
                
            except ValueError:
                continue

            value = float(value)
            graph[from_node].append(to_node)
            val_graph[(from_node, to_node)] = value

            
def dfs(graph, start, end):
    # graph is a hash map {from_node: to_node}
    # start and end are the nodes IDs
    stack = [(start, [])]
    while stack:
        node, path = stack.pop()
        
        if path and node == end:
            yield path
            continue
            
        if node in graph:    
            for next_node in graph[node]:
                if next_node not in path:
                    stack.append((next_node, path + [next_node]))
            

def find_cycles(graph):
    # graph is a hash map {from_node: to_node}
    cycles = []
    
    for node in graph:
        for path in dfs(graph, node, node):
            cycles.append([node] + path)
            
    return cycles


def find_max_cycle(cycles, values):
    # cycles is a list of cycles found using find_cycles(graph)
    # values is a hash map {(from_node, to_node): value}
    max_val = 0
    max_cycle = []
    for path in cycles:
        cur_val = 0
        
        for start in range(len(path) - 1):
            cur_val += values[(path[start], path[start + 1])]
            
        if cur_val > max_val:
            max_val = cur_val
            max_cycle = path
            
    return max_cycle, max_val


build_graph()
cycles = find_cycles(graph)
max_cycle, max_val = find_max_cycle(cycles, val_graph)

In [22]:
print(max_cycle)
print(max_val)

[76, 10, 70, 171, 178, 90, 100, 76]
743000235.0


In [23]:
# Test 1
graph = { 1: [2], 2: [3], 3: [1, 4], 4: [3]}
val_graph = {(1, 2): 1, (2, 3): 3, (3, 4): 1, (3, 1): 5, (4, 3): 10}
cycles = find_cycles(graph)
max_cycle, max_val = find_max_cycle(cycles, val_graph)
print(max_cycle)
print(max_val)

[3, 4, 3]
11


In [24]:
# Test 2
graph = { 1: [2], 2: [3], 3: [4]}
val_graph = {(1, 2): 1, (2, 3): 3, (3, 4): 1}
cycles = find_cycles(graph)
max_cycle, max_val = find_max_cycle(cycles, val_graph)
print(max_cycle)
print(max_val)

[]
0


In [28]:
#!/usr/bin/env python
# encoding: utf-8

import csv
from collections import defaultdict

class Graph(object):
    def __init__(self):
        self.graph = defaultdict(list)
        self.val_graph = defaultdict(list)

        self.build_graph()

    def build_graph(self, filename='../Data/data.csv'):
        with open(filename) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            header = next(csv_reader)
            for row in csv_reader:
                from_node, to_node, value = row

                try:
                    from_node = int(float(from_node))
                    to_node = int(float(to_node))
                except ValueError:
                    continue

                value = float(value)
                self.graph[from_node].append(to_node)
                self.val_graph[(from_node, to_node)] = value

    def dfs(self, graph, start, end):
         # graph is a hash map {from_node: to_node}
        # start and end are the nodes IDs
        stack = [(start, [])]
        while stack:
            node, path = stack.pop()

            if path and node == end:
                yield path
                continue

            if node in graph:
                for next_node in graph[node]:
                    if next_node in path:
                        continue
                    stack.append((next_node, path + [next_node]))

    def find_cycles(self):
        graph = {k : v for k, v in self.graph.items()}
        cycles = []

        for node in graph:
            for path in self.dfs(graph, node, node):
                cycles.append([node] + path)

        return cycles

    def find_max_cycle(self, cycles):
        # cycles is a list of cycles found using find_cycles(graph)
        # values is a hash map {(from_node, to_node): value}
        max_val = 0
        max_cycle = []

        for path in cycles:
            cur_val = 0

            for start in range(len(path) - 1):
                cur_val += self.val_graph[(path[start], path[start + 1])]

            if cur_val > max_val:
                max_val = cur_val
                max_cycle = path

        return max_cycle, max_val

if __name__ == '__main__':
    graph_obj = Graph()
    cycles = graph_obj.find_cycles()
    max_cycle, max_val = graph_obj.find_max_cycle(cycles)
    print("The cycle with the maximum number of nodes is " + str(max_cycle))
    print("The maximum transaction value is %.2f" % max_val)


The cycle with the maximum number of nodes is [76, 10, 70, 171, 178, 90, 100, 76]
The maximum transaction value is 743000235.00
